In [1]:
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [2]:
p_taipei = pd.read_csv("points_taipei.csv")
p_taipei['index'] = range(0, len(p_taipei))

p_new_taipei = pd.read_csv("points_new_taipei.csv")
p_new_taipei['index'] = range(0, len(p_new_taipei))

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)

In [3]:
def hospital_data(file_path):
        df = pd.read_csv(file_path, header=0)
        hos1 = df[df['CATEGORY'].isin(['地區醫院', '區域醫院'])]
        hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf = hospital_count(hos1)
        return hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf
        
def hospital_count(hos):
        # 創建一個包含醫院位置的GeoDataFrame
        hospital_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(hos['POI_X'], hos['POI_Y'])])

        hos_500 = hospital_points1.buffer(500)
        hos_1000 = hospital_points1.buffer(1000)
        hos_2000 = hospital_points1.buffer(2000)

        hos_500_gdf = gpd.GeoDataFrame(geometry=hos_500, crs=hospital_points1.crs)
        hos_1000_gdf = gpd.GeoDataFrame(geometry=hos_1000, crs=hospital_points1.crs)
        hos_2000_gdf = gpd.GeoDataFrame(geometry=hos_2000, crs=hospital_points1.crs)

        return hos_500_gdf, hos_1000_gdf, hos_2000_gdf
    

def overlay_and_count(geometry_gdf, hospital_gdf, prefix):
        intersection = geometry_gdf.overlay(hospital_gdf, how='intersection')
        count_df = intersection.reset_index().groupby('index').size().reset_index(name=prefix + '_cnt')
        return count_df

def calculate_hospital_counts(geometry_gdf, hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf):
        hos1_500_cnt = overlay_and_count(geometry_gdf, hos1_500_gdf, 'hos1_500')
        hos1_1000_cnt = overlay_and_count(geometry_gdf, hos1_1000_gdf, 'hos1_1000')
        hos1_2000_cnt = overlay_and_count(geometry_gdf, hos1_2000_gdf, 'hos1_2000')
        return hos1_500_cnt, hos1_1000_cnt, hos1_2000_cnt

def calculate_weighted_score(df, *args):
        # 進行多個 merge 操作
        for arg in args:
            df = df.merge(arg, on='index', how='left')
        df = df.fillna(0)
        
        # 計算加權分數
        df['醫院加權分數'] = (
            df['hos1_500_cnt'] * 0.5 +
            df['hos1_1000_cnt'] * 0.3 +
            df['hos1_2000_cnt'] * 0.7
        )
        return df


In [4]:
for i in range(2019,2025):
    hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf = hospital_data(f"data_yaer\{i}_hospital.csv")
    
    # 呼叫函式
    hos1_500_cnt_t, hos1_1000_cnt_t, hos1_2000_cnt_t = calculate_hospital_counts(geometry_gdf_t, hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf)
    hos1_500_cnt_nt, hos1_1000_cnt_nt, hos1_2000_cnt_nt = calculate_hospital_counts(geometry_gdf_nt, hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf)

    # 呼叫函式
    result_t = calculate_weighted_score(p_taipei, hos1_500_cnt_t, hos1_1000_cnt_t, hos1_2000_cnt_t)
    result_nt = calculate_weighted_score(p_new_taipei, hos1_500_cnt_nt, hos1_1000_cnt_nt, hos1_2000_cnt_nt)
    # 寫入csv
    result_t.to_csv(f"year_score/{i}_taipei_hos.csv", index=False)
    result_nt.to_csv(f"year_score/{i}_new_taipei_hos.csv", index=False)
    print(i)